In [2]:
import pandas as pd
from datetime import timedelta

In [3]:
# Read in data
ast = pd.read_csv("../Resources/account_state_transitions.csv")

In [4]:
ast.head()

,id,created_when,started_when,account_id,from_state,to_state,previous_state_transition_id
0,3,2020-03-12 20:03:35.724574,2019-11-06 05:49:39.571392,2,CREATED,ENABLED,NaN
1,4,2020-03-12 20:03:35.724574,2019-11-25 22:20:59.150339,2,ENABLED,DISABLED,3.0
2,5,2020-03-12 20:03:36.042456,2019-11-26 10:22:36.571392,2,DISABLED,ENABLED,4.0
3,6,2020-03-12 20:03:36.042456,2019-11-29 11:03:32.150339,2,ENABLED,DISABLED,5.0
4,7,2020-03-12 20:03:36.098448,2019-12-03 10:08:03.571392,2,DISABLED,ENABLED,6.0


## 2) Devise a method of generating a table of all Accounts with their State as of any arbitrary timestamp in the past and the total time they had been in that state as of the timestamp.

### Limit Dataframe by Indicated Timestamp

In [5]:
# Variable to take timestamp input.
timestamp_str = input("Enter date with YYYY-MM-DD format.")

Enter date with YYYY-MM-DD format.2020-03-01


In [6]:
# Convert variable from string to datetime object
timestamp_dt = pd.to_datetime(timestamp_str)

# Adjust timestamp to end of day
timestamp_dt = timestamp_dt + timedelta(hours=23,minutes=59,seconds=59)

In [7]:
# Change data type from string to datetime object
ast['started_when'] = pd.to_datetime(ast['started_when'])

In [8]:
# Create limited dataframe based on indicated timestamp 
df = ast.loc[ast['started_when'] <= timestamp_dt]
df.head()

,id,created_when,started_when,account_id,from_state,to_state,previous_state_transition_id
0,3,2020-03-12 20:03:35.724574,2019-11-06 05:49:39.571392,2,CREATED,ENABLED,NaN
1,4,2020-03-12 20:03:35.724574,2019-11-25 22:20:59.150339,2,ENABLED,DISABLED,3.0
2,5,2020-03-12 20:03:36.042456,2019-11-26 10:22:36.571392,2,DISABLED,ENABLED,4.0
3,6,2020-03-12 20:03:36.042456,2019-11-29 11:03:32.150339,2,ENABLED,DISABLED,5.0
4,7,2020-03-12 20:03:36.098448,2019-12-03 10:08:03.571392,2,DISABLED,ENABLED,6.0


In [9]:
# Select max date for each unique account_id
filtered_df = df.loc[df.groupby('account_id')['started_when'].idxmax()]

# Limit columns
filtered_df = filtered_df[['started_when','account_id','to_state']]

# Reset index
filtered_df = filtered_df.reset_index(drop=True)
filtered_df.head(10)

,started_when,account_id,to_state
0,2020-01-17 13:01:20.571392,1,DISABLED
1,2020-02-24 02:00:21.571392,2,UNLOCKED
2,2020-02-21 11:40:47.887181,3,DISABLED
3,2020-02-25 04:28:59.571392,4,UNLOCKED
4,2020-02-29 20:42:56.571392,5,DISABLED
5,2020-02-20 18:54:47.571392,6,UNLOCKED
6,2020-02-27 21:56:59.571392,7,ENABLED
7,2020-02-27 05:59:01.834550,8,DISABLED
8,2020-02-28 09:49:07.360866,9,DISABLED
9,2020-02-21 15:48:49.571392,10,UNLOCKED


### Total Time in State as of Indicated Timestamp

In [10]:
# List comprehension: find difference between timestamp_dt and started_when 
time_in_state = [timestamp_dt - x for x in filtered_df['started_when']]

# Add column to dataframe
filtered_df[f'time_in_state_as_of_{timestamp_str[:10]}'] = time_in_state

# Preview dataframe
print(f"TOTAL TIME IN CURRENT STATE BY ACCOUNT AS OF: {timestamp_str[:10]}")
filtered_df.head()

TOTAL TIME IN CURRENT STATE BY ACCOUNT AS OF: 2020-03-01


,started_when,account_id,to_state,time_in_state_as_of_2020-03-01
0,2020-01-17 13:01:20.571392,1,DISABLED,44 days 10:58:38.428608
1,2020-02-24 02:00:21.571392,2,UNLOCKED,6 days 21:59:37.428608
2,2020-02-21 11:40:47.887181,3,DISABLED,9 days 12:19:11.112819
3,2020-02-25 04:28:59.571392,4,UNLOCKED,5 days 19:30:59.428608
4,2020-02-29 20:42:56.571392,5,DISABLED,1 days 03:17:02.428608


In [11]:
# Export to csv
filtered_df.to_csv(f"../Output/time_in_state_as_of_{timestamp_str[:10]}.csv", index=False)